# Extract structured data from images

Use AI vision to extract JSON data from receipts, forms, documents, and other images.

## Problem

You have images containing structured information (receipts, forms, ID cards) and need to extract specific fields as JSON for downstream processing.

| Image | Fields to extract |
|-------|------------------|
| receipt.jpg | vendor, total, date, items |
| business_card.jpg | name, email, phone, company |
| invoice.pdf | invoice_number, amount, due_date |

## Solution

**What's in this recipe:**
- Extract structured JSON from images using GPT-4o
- Use `openai.vision()` which handles images directly
- Access individual fields from the extracted data

You use Pixeltable's `openai.vision()` function which automatically handles image encoding. Request JSON output via `response_format` in `model_kwargs`.

### Setup

In [1]:
%pip install -qU pixeltable openai

import os
import getpass

if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key: ')

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pixeltable as pxt
from pixeltable.functions import openai

### Load images

In [3]:
# Create a fresh directory
pxt.drop_dir('extraction_demo', force=True)
pxt.create_dir('extraction_demo')

Connected to Pixeltable database at: postgresql+psycopg://postgres:@/pixeltable?host=/Users/pjlb/.pixeltable/pgdata
Created directory 'extraction_demo'.


In [4]:
t = pxt.create_table('extraction_demo.images', {'image': pxt.Image})

Created table 'images'.


In [5]:
# Insert sample images
t.insert([
    {'image': 'https://raw.githubusercontent.com/pixeltable/pixeltable/main/docs/resources/images/000000000036.jpg'},
    {'image': 'https://raw.githubusercontent.com/pixeltable/pixeltable/main/docs/resources/images/000000000090.jpg'},
])

Error: Not a valid image: /Users/pjlb/.pixeltable/file_cache/8fd441aba65942ab89674c1e5e4eda12_0_a8b594abb60e6eda0156bbfa8479de2771ada2268fb17f999c18904326e9a762.jpg

### Extract structured data

Use `openai.vision()` to analyze images and get JSON output:

In [ ]:
# Add extraction column using openai.vision (handles images directly)
PROMPT = '''Analyze this image and extract the following as JSON:
- description: A brief description of the image
- objects: List of objects visible in the image
- dominant_colors: List of dominant colors
- scene_type: Type of scene (indoor, outdoor, etc.)'''

t.add_computed_column(
    data=openai.vision(
        prompt=PROMPT,
        image=t.image,
        model='gpt-4o-mini',
        model_kwargs={'response_format': {'type': 'json_object'}}
    )
)

KeyboardInterrupt: 

In [ ]:
# View extracted data
t.select(t.image, t.data).collect()

In [ ]:
# You can also parse the JSON into individual columns if needed
import json

@pxt.udf
def parse_description(data: str) -> str:
    return json.loads(data).get('description', '')

t.select(t.image, description=parse_description(t.data)).collect()

## Explanation

**Why use `openai.vision()`:**

- Handles PIL Images directly (no manual base64 encoding or URL storage)
- Simpler API than constructing chat messages manually
- Returns the content string directly

**Getting JSON output:**

Pass `model_kwargs={'response_format': {'type': 'json_object'}}` to get structured JSON.

**Other extraction use cases:**

| Use Case | Fields to extract |
|----------|------------------|
| Receipts | vendor, total, date, items, tax |
| Business cards | name, title, company, email, phone |
| Product photos | brand, model, condition, defects |
| Documents | title, date, author, summary |

## See also

- [Analyze images in batch](https://docs.pixeltable.com/howto/cookbooks/images/vision-batch-analysis)
- [Configure API keys](https://docs.pixeltable.com/howto/cookbooks/core/workflow-api-keys)